# Transfer Learning - Cats vs Dogs

In [ ]:
# Imports
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

## Carregando o dataset Cats vs Dogs

In [ ]:
IMG_SIZE = 160
(ds_train, ds_val), ds_info = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:]'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

def format_example(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
    return image, label

ds_train = ds_train.map(format_example).batch(32).prefetch(tf.data.AUTOTUNE)
ds_val = ds_val.map(format_example).batch(32).prefetch(tf.data.AUTOTUNE)

## Criando o modelo com MobileNetV2

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3),
                                                 include_top=False,
                                                 weights='imagenet')
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1, activation='sigmoid')
])

## Compilando e treinando

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(ds_train, validation_data=ds_val, epochs=1)

## Avaliação do modelo

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

plt.plot([1], acc, 'bo', label='Treinamento')
plt.plot([1], val_acc, 'ro', label='Validação')
plt.legend()
plt.title('Acurácia por Época')
plt.xlabel('Época')
plt.ylabel('Acurácia')
plt.show()